# <span style="color:red"> Main Modelling - Bagging - Gridsearch </span>
* read in pickle
* keep 3 translators
* final selected features based on initial analysis with logistic regression
* drop 'A Wild Sheep Chase' due to unique text
* run bagging grid search with dt and knn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import textacy
import re
import pickle
import os

from collections import Counter

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# extend limit of number of rows and columns to display in cell
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100


Bad key "text.kerning_factor" on line 4 in
/Users/Steven/opt/anaconda3/envs/textacy/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### Load the dataframe containing text chunks and related variables

In [2]:
in_full_path = '../../../Documents/murakami/pkl3/df_all_v02.pkl'
# read back pickle
with open (in_full_path, 'rb') as fp:
    df = pickle.load(fp)

### Check the dataframe is as expected

In [3]:
df.head(1)

,book_chunk_no,number,title,text,fstop_indices,split_indices,chunks,translator,book_title,parsed,n_sents,n_words,n_chars,n_syllables,n_unique_words,n_long_words,n_monosyllable_words,n_polysyllable_words,trans_code,chunk_length,n_sents_norm,n_words_norm,n_chars_norm,n_syllables_norm,n_unique_words_norm,n_long_words_norm,n_monosyllable_words_norm,n_polysyllable_words_norm,vader_compound,vader_neg,vader_neu,vader_pos,pron_count,verb_count,det_count,adj_count,num_count,punct_count,noun_count,adp_count,cconj_count,sconj_count,adv_count,aux_count,part_count,propn_count,space_count,intj_count,sym_count,x_count,...,where_adv,too_adv,now_adv,all_adv,then_adv,just_adv,really_adv,there_adv,only_adv,even_adv,as_adv,always_adv,when_adv,again_adv,here_adv,maybe_adv,still_adv,back_adv,why_adv,very_adv,never_adv,how_adv,whole_adj,small_adj,long_adj,able_adj,more_adj,sure_adj,big_adj,last_adj,bad_adj,old_adj,same_adj,good_adj,new_adj,only_adj,different_adj,other_adj,next_adj,few_adj,much_adj,hard_adj,real_adj,young_adj,own_adj,first_adj,deep_adj,strange_adj,little_adj,right_adj
0,0,1,Wednesday Afternoon Picnic,IT WAS A short one-paragraph item in the morn...,"[57, 97, 115, 196, 318, 385, 420, 445, 504, 65...","[967, 1924, 2998, 3982, 4935, 5975, 6995, 7961...",IT WAS A short one-paragraph item in the morni...,Alfred Birnbaum,A Wild Sheep Chase,"(IT, WAS, A, short, one, -, paragraph, item, i...",15,174,742,240,116,33,128,17,0,944,15.889831,184.322034,786.016949,254.237288,122.881356,34.957627,135.59322,18.008475,-0.4798,0.075,0.862,0.064,18.0,20.0,31.0,9.0,2.0,33.0,51.0,19.0,6.0,3.0,5.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df.shape

(5253, 141)

In [5]:
# df.columns

### Drop 'A Wild Sheep Chase' 

In [6]:
df = df[df['book_title'] != 'A Wild Sheep Chase']

### Check the baseline

In [7]:
baseline_acc = df['trans_code'].value_counts(normalize=True).max()
baseline_acc

0.4364282726892514

### Select Predictor Variables

In [8]:
def feature_select(basic_counts=True, vader=False, pos_counts=False, 
                   words=False, adv=False, adj=False):
    '''create column list depending on features to include in the modelling'''
    columns = []
    if basic_counts:
        columns += [i for i in df.columns if i.startswith('n_') & i.endswith('_norm')]
    if vader:
        columns += [i for i in df.columns if i.startswith('vader_')]
    if pos_counts:
        columns += [i for i in df.columns if i.endswith('_count_norm')]
    if words:
        columns += [i for i in df.columns if i.endswith('_w')]
    if adj:
        columns += [i for i in df.columns if i.endswith('_adj')]
    if adv:
        columns += [i for i in df.columns if i.endswith('_adv')]
    return columns

In [9]:
cols_to_drop = ['num_count_norm', 'punct_count_norm','space_count_norm', 'sym_count_norm', 'x_count_norm']
predictor_cols = feature_select(basic_counts=True, pos_counts=True, vader=False, words=True, adj=True, adv=True)
predictor_cols = [x for x in predictor_cols if x not in cols_to_drop]

### Set Target and Predictor variables

In [10]:
X = df[predictor_cols]
y = df['trans_code']

### Make Train Test Split with stratify on y

In [11]:
# stratify based on your target variable
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=1)

### Standardise the predictor variables

In [12]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

### Bagging classification with cross validation

In [13]:
dt = DecisionTreeClassifier(max_depth=None)

In [14]:
bg_dt_01 = BaggingClassifier(base_estimator=dt, n_estimators=300)

In [15]:
bg_dt_01.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [16]:
# check the score on the train data
print('train score:', bg_dt_01.score(X_train, y_train))
print('CV score:', cross_val_score(bg_dt_01, X_train, y_train, cv=5).mean())
# check the score on the test data
print('test score:', bg_dt_01.score(X_test, y_test))

train score: 1.0
CV score: 0.6596078431372548
test score: 0.6854754440961337


#### bg with dt does better than KNN, rf or  DT on its own for the moment. It is not as good as logistic regression though

### Confusion Matrix

In [17]:
predictions = bg_dt_01.predict(X_test)

In [18]:
conmat = confusion_matrix(
    y_test, predictions, labels=[0, 1, 2])

confusion = pd.DataFrame(conmat, index=['actual birnbaum', 'actual rubin', 'actual gabriel'],
                         columns=['predicted birnbaum', 'predicted rubin', 'predicted gabriel'])
confusion

,predicted birnbaum,predicted rubin,predicted gabriel
actual birnbaum,126,70,62
actual rubin,36,342,40
actual gabriel,32,61,188


* Similar to lr but not quite as good 

### Classification Report

In [19]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.65      0.49      0.56       258
           1       0.72      0.82      0.77       418
           2       0.65      0.67      0.66       281

    accuracy                           0.69       957
   macro avg       0.67      0.66      0.66       957
weighted avg       0.68      0.69      0.68       957



### Bagging with DT and gridsearch

In [20]:
model = BaggingClassifier(base_estimator=dt, n_estimators=100)
# params = {'max_samples': np.linspace(0.8, 1.0, 3),
#           'max_features': range(int(3/4.*X.shape[1]), X.shape[1]+1)}
params = {'n_estimators': [100, 200, 300],
          'max_samples': [0.5, 0.8],
          'max_features': [0.5, 0.8]}



bg_dt_gs_01 = GridSearchCV(model, 
                           param_grid=params, 
                           cv=5, 
                           n_jobs=2,
                           verbose=1)
bg_dt_gs_01.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  2.2min
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:  3.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                               class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                         

In [21]:
def model_gridsearch_score(model, X_train, y_train, X_test, y_test):
    
    # print the grid search results
    results_dict = {}
    print('Best Parameters:')
    print(model.best_params_)
    print('Best estimator mean cross validated training score:')
    print(model.best_score_)
    print('Best estimator score on the full training set:')
    print(model.score(X_train, y_train))
    print('Best estimator score on the test set:')
    print(model.score(X_test, y_test))
    print('ROC-AUC score on the test set:')
    
    # store the grid search results as a dictionary
    y_bin = label_binarize(y_test, model.classes_)
    for i, class_ in enumerate(model.classes_):
        print('Class {}:'.format(class_), round(roc_auc_score(y_bin[:,i],model.predict_proba(X_test)[:,i]),2))
    results_dict['Best_Parameters'] = model.best_params_
    results_dict['Best_CV_Score'] = model.best_score_
    results_dict['Best_Train_Score'] = model.score(X_train, y_train)
    results_dict['Best_Test_Score'] = model.score(X_test, y_test)
    results_dict['AUC_Class_0'] = roc_auc_score(y_bin[:,0],model.predict_proba(X_test)[:,0])
    results_dict['AUC_Class_1'] = roc_auc_score(y_bin[:,1],model.predict_proba(X_test)[:,1])
    results_dict['AUC_Class_2'] = roc_auc_score(y_bin[:,2],model.predict_proba(X_test)[:,2])
    predictions = model.predict(X_test)
    results_dict['conmat'] = confusion_matrix(
        y_test, predictions, labels=[0, 1, 2])

    return results_dict

In [24]:
bg_dt_gs_01_results = model_gridsearch_score(bg_dt_gs_01, X_train, y_train, X_test, y_test)

Best Parameters:
{'max_features': 0.8, 'max_samples': 0.5, 'n_estimators': 300}
Best estimator mean cross validated training score:
0.6687581699346404
Best estimator score on the full training set:
0.9947712418300654
Best estimator score on the test set:
0.670846394984326
ROC-AUC score on the test set:
Class 0: 0.8
Class 1: 0.86
Class 2: 0.85


### Save the model

In [25]:
# set the output path and name
out_path = '../../../Documents/murakami/pkl_models/'
out_name = 'bg_dt_gs_01_ns'
out_full_path = out_path + out_name + '.pkl'

# save pickle
with open(out_full_path, 'wb') as fp:
    pickle.dump(bg_dt_gs_01, fp)

### Confusion Matrix

In [26]:
predictions = bg_dt_gs_01.predict(X_test)

In [27]:
conmat = confusion_matrix(
    y_test, predictions, labels=[0, 1, 2])

confusion = pd.DataFrame(conmat, index=['actual birnbaum', 'actual rubin', 'actual gabriel'],
                         columns=['predicted birnbaum', 'predicted rubin', 'predicted gabriel'])
confusion

,predicted birnbaum,predicted rubin,predicted gabriel
actual birnbaum,127,80,51
actual rubin,39,348,31
actual gabriel,32,82,167


Very slight improvement for bagging with dt and with gridsearch not not significant 

### Bagging with KNN

In [28]:
knn = KNeighborsClassifier()
model = BaggingClassifier(base_estimator=knn)
params = {'n_estimators': [100, 200, 300],
          'max_samples': [0.5, 0.8],
          'max_features': [0.5, 0.8]}



bg_knn_gs_01 = GridSearchCV(model, 
                            param_grid=params, 
                            cv=5, 
                            n_jobs=2,
                            verbose=1)
bg_knn_gs_01.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  9.9min
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed: 16.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                                             leaf_size=30,
                                                                             metric='minkowski',
                                                                             metric_params=None,
                                                                             n_jobs=None,
                                                                             n_neighbors=5,
                                                                             p=2,
                                                                             weights='uniform'),
                                         bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                       

In [30]:
bg_knn_gs_01_results = model_gridsearch_score(bg_knn_gs_01, X_train, y_train, X_test, y_test)

Best Parameters:
{'max_features': 0.5, 'max_samples': 0.8, 'n_estimators': 200}
Best estimator mean cross validated training score:
0.5790849673202614
Best estimator score on the full training set:
0.8682352941176471
Best estimator score on the test set:
0.5768025078369906
ROC-AUC score on the test set:
Class 0: 0.76
Class 1: 0.8
Class 2: 0.81


### Save the model

In [32]:
# set the output path and name
out_path = '../../../Documents/murakami/pkl_models/'
out_name = 'bg_knn_gs_01_ns'
out_full_path = out_path + out_name + '.pkl'

# save pickle
with open(out_full_path, 'wb') as fp:
    pickle.dump(bg_knn_gs_01, fp)

### Confusion Matrix

In [33]:
predictions = bg_knn_gs_01.predict(X_test)

In [34]:
conmat = confusion_matrix(
    y_test, predictions, labels=[0, 1, 2])

confusion = pd.DataFrame(conmat, index=['actual birnbaum', 'actual rubin', 'actual gabriel'],
                         columns=['predicted birnbaum', 'predicted rubin', 'predicted gabriel'])
confusion

,predicted birnbaum,predicted rubin,predicted gabriel
actual birnbaum,92,134,32
actual rubin,37,359,22
actual gabriel,39,141,101


### Classification Report

In [ ]:
print(classification_report(y_test, predictions))